pyJHTDB are failed to compile on windows. One alternative way might be to use [zeep](https://python-zeep.readthedocs.io/en/master/) package.    
More details can be found at [http://test.turbulence.pha.jhu.edu/service/turbulence.asmx](http://test.turbulence.pha.jhu.edu/service/turbulence.asmx)

In [1]:
import zeep
import numpy as np

client = zeep.Client('http://test.turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')

token="edu.jhu.pha.turbulence.testing-201406" #replace with your own token

nnp=5 #number of points
points=np.random.rand(nnp,3)

# convert to JHTDB structures
x_coor=ArrayOfFloat(points[:,0].tolist())
y_coor=ArrayOfFloat(points[:,1].tolist())
z_coor=ArrayOfFloat(points[:,2].tolist())
point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);

print(points)

[[0.04330301 0.26989882 0.30953609]
 [0.55136145 0.26644196 0.55143142]
 [0.18038822 0.69407639 0.1385497 ]
 [0.29323826 0.38428839 0.31845546]
 [0.87714551 0.22212711 0.75334416]]


#### In `GetData_Python`, Function_name could be  

GetVelocity,            GetMagneticField,          GetVectorPotential,    
GetVelocityGradient,    GetMagneticFieldGradient,  GetVectorPotentialGradient,    
GetVelocityHessian,     GetMagneticHessian,        GetVectorPotentialHessian,    
GetVelocityLaplacian,   GetMagneticFieldLaplacian, GetVectorPotentialLaplacian,    

GetPressure,            GetTemperature,            GetDensity,    
GetPressureGradient,    GetTemperatureGradient,    GetDensityGradient,    
GetPressureHessian,     GetTemperatureHessian,     GetDensityHessian,     

GetVelocityAndPressure, GetVelocityAndTemperature, GetForce, GetInvariant

In [2]:
Function_name="GetVelocityGradient" 
time=0.6
number_of_component=9 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetData_Python(Function_name, token,"isotropic1024coarse", 0.6, 
                                     SpatialInterpolation("None_Fd4"), TemporalInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))

print(result)

[[ -3.555322    -2.0177238    2.885566    -3.031634     1.08202827
    1.263689     2.67636633   1.19449687   2.60499573]
 [  9.063015    39.41201      9.581824   -15.5420218  -16.65086
   12.1881762   -8.755374   -28.9169388    8.073001  ]
 [  3.78538179   2.43797469 -13.5845661    0.90120286  -3.36181474
   -5.24289131  13.7902565    2.47591329  -0.44196767]
 [  1.9354825   -7.31927061  -5.513446     0.55677354   5.47382641
   -3.92193151   9.435512    19.7658043   -7.49285936]
 [  6.72795773  -3.45526481   9.70396      7.839298     4.58286572
    1.574099     3.225538     5.076862   -11.4413824 ]]


#### In `GetPosition_Python`, Function_name could be
GetPosition only.

In [3]:
Function_name="GetPosition"
startTime=0.1
endTime=0.2
dt=0.02
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetPosition_Python(Function_name, token,"isotropic1024coarse", startTime, endTime, dt, 
                                         SpatialInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[0.04323319 0.15412635 0.21834722]
 [0.59860975 0.10928211 0.56112105]
 [0.10606352 0.5664194  0.10957711]
 [0.32639155 0.2539691  0.27564546]
 [0.9109745  0.21388906 0.6731475 ]]


#### In `GetFilter_Python`, Function_name could be    

GetBoxFilter, GetBoxFilterSGSscalar, GetBoxFilterSGSvector,     
GetBoxFilterSGSsymtensor, GetBoxFilterSGStensor, GetBoxFilterGradient.

In [4]:
Function_name="GetBoxFilter" #could also be
field="u"
time=0.6
filterwidth=0.05
spacing=0 #spacing is only used in GetBoxFilterGradient, but always provide it.
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetFilter_Python("GetBoxFilter",token,"isotropic1024coarse", field, 
                                       time, filterwidth, SpatialInterpolation("None"), point, spacing) 
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[-0.17953077 -1.30879486  0.19406827]
 [ 0.07615118 -1.18771946 -0.6653834 ]
 [-0.63668466 -1.259746    0.18242529]
 [-0.61741746 -1.72727871 -0.10093431]
 [ 0.2955498   0.14606948 -1.02642322]]


In [5]:
import struct
import base64

field="u"
timestep=1
x_start=1
y_start=1
z_start=1
x_end=2
y_end=5
z_end=8
x_step=1
y_step=1
z_step=1
filter_width=0
    
result=client.service.GetAnyCutoutWeb(token,"isotropic1024coarse", field, timestep,
                                       x_start, y_start, z_start, x_end, y_end, z_end,
                                      x_step, y_step, z_step, filter_width, "")  # put empty string for the last parameter

# transfer base64 format to numpy
number_of_component=3 # change this based on the field
nx=len(range(x_start, x_end+1, x_step))
ny=len(range(y_start, y_end+1, y_step))
nz=len(range(z_start, z_end+1, z_step))
base64_len=int(nx*ny*nz*number_of_component)
base64_format='<'+str(base64_len)+'f'

result=struct.unpack(base64_format, result)
result=np.array(result).reshape((nz, ny, nx, number_of_component))
print(result.shape)  # see the shape of the result and compare it with nx, ny, nz and number of component

(8, 5, 2, 3)
